# 50. データの入手・整形

In [1]:
import pandas as pd
from random import shuffle

In [11]:
'''
出来ないが？
'''
col_names = ['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP']
df = pd.read_csv("./data/NewsAggregatorDataset/newsCorpora.csv", encoding = 'UTF-8', names = col_names, sep = '\t')
extracted_data_frame = df[df['PUBLISHER'].isin(['Reuters', 'Huffington Post', 'Businessweek', 'Contactmusic.com', 'Daily Mail'])][['TITLE', 'CATEGORY']]
row_size, _ = extracted_data_frame.shape
data_list = [{} for _ in range(row_size)]

for data_name in extracted_data_frame:
    for idx, frame in enumerate(extracted_data_frame[data_name]):
        data_list[idx][data_name] = frame


shuffle(data_list)

train_size = row_size // 10 * 8
valid_size = row_size // 10
test_size = row_size // 10

path = './work/'
file_names = ['train.txt', 'valid.txt', 'test.txt']
data_names = ['train_data', 'valid_data', 'test_data']

file_data = dict()
file_data['train_data'] = data_list[:train_size]
file_data['valid_data'] = data_list[train_size:train_size + valid_size]
file_data['test_data'] = data_list[-test_size:]


for idx, file_name in enumerate(file_names):
    with open(path + file_name, mode = 'w') as f:
        for data in file_data[data_names[idx]]:
            f.write('\t'.join([data['TITLE'], data['CATEGORY']]) + '\n')

In [7]:
data_list = list()
with open('./data/NewsAggregatorDataset/newsCorpora.csv', mode = 'r') as f:
    for line in f:
        splited_line = line.split('\t')
        if splited_line[3] in ['Reuters', 'Huffington Post', 'Businessweek', 'Contactmusic.com', 'Daily Mail']:
            data_list.append([splited_line[1], splited_line[4]])

row_size = len(data_list)
shuffle(data_list)

train_size = row_size // 10 * 8
valid_size = row_size // 10
test_size = row_size // 10

path = './work/'
file_names = ['train.txt', 'valid.txt', 'test.txt']
data_names = ['train_data', 'valid_data', 'test_data']

file_data = dict()
train_data = data_list[:train_size]
valid_data = data_list[train_size:train_size + valid_size]
test_data = data_list[-test_size:]

file_data['train_data'] = train_data
file_data['valid_data'] = valid_data 
file_data['test_data'] = test_data

for idx, file_name in enumerate(file_names):
    with open(path + file_name, mode = 'w') as f:
        for data in file_data[data_names[idx]]:
            f.write('\t'.join([data[0], data[1]]) + '\n')

In [8]:
!wc ./work/*

  1335  15216  89224 ./work/test.txt
 10680 122450 713624 ./work/train.txt
  1335  15247  89063 ./work/valid.txt
 13350 152913 891911 total


# 51. 特徴量抽出

0 : ビジネス  
1 : 科学技術  
2 : エンターテイメント  
3 : 健康  

In [69]:
def save_file_npy(dir_name, file_name, x):
    np.save(f'{dir_name}{file_name}', x)
        
def load_file_npy(dir_name, file_name):
    return np.load(f'{dir_name}{file_name}').astype(np.float32)

def chr2num(y):
    converter = dict()
    converter['b'] = 0
    converter['t'] = 1
    converter['e'] = 2
    converter['m'] = 3
    return np.asarray([converter[article_type] for article_type in y])

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import numpy as np

In [35]:
vectorizer = TfidfVectorizer()
train_tfidf = vectorizer.fit_transform(list(map(lambda x: x[0], train_data)))

In [42]:
print('Vocabulary size: {}'.format(train_tfidf.shape[1]))

Vocabulary size: 12829


In [67]:
pd.DataFrame(train_tfidf.toarray(), columns = vectorizer.get_feature_names())

,00,07,08,09,0ff,0ut,10,100,1000,10000,...,zoe,zombie,zombies,zone,zooey,zuckerberg,zynga,zâ,œf,œwaist
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10675,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10676,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10677,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10678,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
valid_tfidf = vectorizer.transform(list(map(lambda x: x[0], valid_data)))
test_tfidf = vectorizer.transform(list(map(lambda x: x[0], test_data)))

<class 'scipy.sparse.csr.csr_matrix'>


In [75]:
save_file_npy('work/', 'train_feature', train_tfidf.toarray())
save_file_npy('work/', 'valid_feature', valid_tfidf.toarray())
save_file_npy('work/', 'test_feature', test_tfidf.toarray())

In [79]:
train_y = chr2num(list(map(lambda x: x[1], train_data)))
valid_y = chr2num(list(map(lambda x: x[1], valid_data)))
test_y = chr2num(list(map(lambda x: x[1], test_data)))

In [80]:
save_file_npy('work/', 'train_y', train_y)
save_file_npy('work/', 'valid_y', valid_y)
save_file_npy('work/', 'test_y', test_y)

# 52. 学習

In [82]:
train_x = load_file_npy('work/', 'train_feature.npy')
train_y = load_file_npy('work/', 'train_y.npy')
valid_x = load_file_npy('work/', 'valid_feature.npy')
valid_y = load_file_npy('work/', 'valid_y.npy')
test_x = load_file_npy('work/', 'test_feature.npy')
test_y = load_file_npy('work/', 'test_y.npy')

In [77]:
from sklearn.linear_model import LogisticRegression

In [83]:
lr = LogisticRegression()
lr.fit(train_x, train_y)

/Users/y_nakamura/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/y_nakamura/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

# 53. 予測

In [88]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [93]:
train_pred_y = lr.predict(train_x)
test_pred_y = lr.predict(test_x)

In [94]:
print(test_pred_y)

[2. 3. 0. ... 2. 2. 0.]


# 54. 正解率の計測

In [97]:
print('train accuracy = {}'.format(accuracy_score(y_true = train_y, y_pred = train_pred_y)))
print('test accuracy = {}'.format(accuracy_score(y_true = test_y, y_pred = test_pred_y)))

train accuracy = 0.9218164794007491
test accuracy = 0.8816479400749063


# 55. 混同行列の作成

In [100]:
print('train confusion matrix = \n', confusion_matrix(y_true = train_y, y_pred = train_pred_y))

train confusion matrix = 
 [[4370   32   85    3]
 [ 223  805  170    2]
 [  24    4 4211    0]
 [ 105    4  183  459]]


In [101]:
print('test confusion matrix = \n', confusion_matrix(y_true = test_y, y_pred = test_pred_y))

test confusion matrix = 
 [[538   6  18   1]
 [ 39  92  32   2]
 [ 12   2 511   0]
 [ 23   1  22  36]]


# 56. 適合率，再現率，F1スコアの計測

これ自信ないぽよ

In [117]:
precision_micro = precision_score(y_true = test_y, y_pred = test_pred_y, average = 'micro')
print('precision micro = ', precision_micro)
precision_macro = precision_score(y_true = test_y, y_pred = test_pred_y, average = 'macro')
print('precision macro = ', precision_macro)
recall_micro = recall_score(y_true = test_y, y_pred = test_pred_y, average = 'micro')
print('precision = ', (precision_micro + precision_macro) / 2)
print('recall micro = ', recall_micro)
recall_macro = recall_score(y_true = test_y, y_pred = test_pred_y, average = 'macro')
print('recall macro = ', recall_macro)
print('recall = ', (recall_micro + recall_macro) / 2)
f1_score_micro = f1_score(y_true = test_y, y_pred = test_pred_y, average = 'micro')
print('f1 score micro = ', f1_score_micro)
f1_score_macro = f1_score(y_true = test_y, y_pred = test_pred_y, average = 'macro')
print('f1 score macro = ', f1_score_macro)
print('f1 score = ', (f1_score_micro + f1_score_macro) / 2)

precision micro =  0.8816479400749063
precision macro =  0.8973884592847572
precision =  0.8895181996798318
recall micro =  0.8816479400749063
recall macro =  0.7313821269489943
recall =  0.8065150335119503
f1 score micro =  0.8816479400749062
f1 score macro =  0.7812244994883822
f1 score =  0.8314362197816443


# 57. 特徴量の重みの確認

In [129]:
print(lr.coef_)

[[-3.36767703 -3.18083715 -3.10623132 ...  4.40239917  4.57616196
   4.62608361]
 [-0.03444174 -0.0186113  -0.01232489 ... -0.0317544  -0.03314261
  -0.02191253]
 [-0.15160942 -0.06559459 -0.02614219 ...  0.04693901  0.14384818
   0.0439173 ]
 [-0.0285663  -0.01598717 -0.01064402 ... -0.02231331 -0.01760548
  -0.01495018]]


In [146]:
feature_names = np.array(vectorizer.get_feature_names())
print(feature_names[:10])

['00' '07' '08' '09' '0ff' '0ut' '10' '100' '1000' '10000']


In [169]:
#各カテゴリの重み最大の単語
max_indices = np.argpartition(-lr.coef_, 10)[:,:10]
print(feature_names[max_indices])

[['zynga' 'zâ' 'œwaist' 'zooey' 'zombies' 'zone' 'œf' 'zuckerberg'
  'zombie' 'zoe']
 ['facebook' 'apple' 'google' 'microsoft' 'tesla' 'gm' 'climate'
  'samsung' 'comcast' 'nasa']
 ['she' 'kardashian' 'star' 'kim' 'chris' 'her' 'miley' 'cyrus' 'film'
  'movie']
 ['study' 'mers' 'drug' 'ebola' 'cdc' 'health' 'cancer' 'fda' 'could'
  'cases']]


In [170]:
#各カテゴリの重み最小の単語
min_indices = np.argpartition(lr.coef_, 10)[:,:10]
print(feature_names[min_indices])

[['07' '00' '08' '09' '0ff' '0ut' '100' '10' '1000' '10000']
 ['kim' 'ecb' 'fed' 'bank' 'her' 'kardashian' 'euro' 'stocks' 'ukraine'
  'thrones']
 ['billion' 'says' 'fed' 'bank' 'ceo' 'china' 'facebook' 'us' 'google'
  'update']
 ['bank' 'google' 'fed' 'kim' 'kardashian' 'china' 'stocks' 'ecb' 'euro'
  'gm']]


# 58. 正則化パラメータの変更

# 59. ハイパーパラメータの探索